<a href="https://colab.research.google.com/github/Rainniee/Neural-Networks-AI/blob/master/Assignment_2_2_AutoKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Autokeras

In [2]:
!pip install -q autokeras

    100% |████████████████████████████████| 102kB 5.4MB/s 
    100% |████████████████████████████████| 26.6MB 1.2MB/s 
    100% |████████████████████████████████| 560.1MB 24kB/s 
    100% |████████████████████████████████| 13.9MB 2.6MB/s 
    100% |████████████████████████████████| 25.3MB 1.4MB/s 
    100% |████████████████████████████████| 51kB 21.0MB/s 
    100% |████████████████████████████████| 83.1MB 561kB/s 
    100% |████████████████████████████████| 61kB 21.8MB/s 
    100% |████████████████████████████████| 8.9MB 2.7MB/s 
    100% |████████████████████████████████| 1.6MB 6.2MB/s 
    100% |████████████████████████████████| 1.4MB 4.7MB/s 
    100% |████████████████████████████████| 143kB 29.1MB/s 
    100% |████████████████████████████████| 25.4MB 1.1MB/s 
    100% |████████████████████████████████| 2.0MB 5.4MB/s 
    100% |████████████████████████████████| 706kB 7.2MB/s 
thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
pymc

In [3]:
# autokeras
import autokeras as ak
from autokeras.nn.metric import Accuracy
from autokeras.image.image_supervised import load_image_dataset, ImageClassifier

# others
import pandas as pd
import pickle

from keras.models import load_model
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping

import shutil
import PIL
import tensorflow as tf

from time import time

tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


### Load tiny-imagenet-200

In [11]:
from google.colab import files
uploaded = files.upload()

Saving val_images.pkl to val_images.pkl


In [0]:
train_labels = pickle.load( open( 'train_labels.pkl', 'rb' ) )

In [0]:
train_images = pickle.load( open( 'train_images.pkl', 'rb') )

In [0]:
val_labels = pickle.load( open( 'val_labels.pkl', 'rb' ) )

In [0]:
val_images = pickle.load( open( 'val_images.pkl', 'rb') )

### Build Image Classifier

In [13]:
train_images = train_images.astype('float32')
val_images = val_images.astype('float32')
train_images /= 255
val_images /= 255

print('train data shape:', train_images.shape)
print('val data shape:', val_images.shape)

train data shape: (100000, 32, 32, 3)
val data shape: (10000, 32, 32, 3)


In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)

num_classes = 200

start = time()

clf = ImageClassifier(verbose=True, augment=True)

clf.fit(train_images, train_labels, time_limit=8 * 60 * 60)

# let it retrain on the best model to train
clf.final_fit(train_images, train_labels, val_images, val_labels, retrain=True)

# Given a path and save the weights
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'autokeras_imagenet200.h5'

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)

Saving Directory: /tmp/autokeras_JXE7JU
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
Epoch-1, Current Metric - 0:  15%|████▎                       | 120/778 [01:06<06:21,  1.72 batch/s]

In [0]:
train_summary = clf.evaluate(train_images, train_labels)
val_summary = clf.evaluate(val_images, val_labels)

print('Training Accuracy: ' + str(train_summary))
print('Validation Accuracy: ' + str(val_summary))

In [0]:
# Save the model
# clf.export_autokeras_model(model_path)
model_dir = os.path.join(save_dir, 'autokeras_models_keras_imagenet200.h5')
clf.export_keras_model(model_dir)

In [0]:
model = load_model('saved_models/autokeras_models_keras_imagenet200.h5') 

model.summary()

#autokeras_model = ak.utils.pickle_from_file(my_model_path)
#keras_model = autokeras_model.graph.produce_keras_model()

In [0]:
from keras.utils import plot_model
plot_model(model)

In [0]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))